In [28]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import requests
import zipfile
import io
import os
import os
import io
import zipfile
import requests
import re
from datetime import datetime

In [29]:
# current working directory
cwd = os.getcwd()
#set wroking direvtly to one level up
os.chdir(os.path.dirname(cwd))
# rpint new working directory
print("Current working directory:", os.getcwd())

Current working directory: /


## Please update the URL swith the lastest ons change file data

In [ ]:
# === Step 1: Define download URL and output path ===
url = "https://www.arcgis.com/sharing/rest/content/items/3acc892515aa49a8885c2deb734ebd3d/data"
output_dir = os.path.join("gsscoder_python", "lookups")
os.makedirs(output_dir, exist_ok=True)

# === Step 2: Download the ZIP file ===
response = requests.get(url)
if not response.ok:
    raise Exception(f"Download failed with status code {response.status_code}")

# === Step 3: Extract only the relevant CSV files ===
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    # Identify any CSVs and try to extract the database name
    extracted_files = []
    zip_file_names = z.namelist()

    # Try to identify the name of the database file for date extraction
    db_file_name = next((f for f in zip_file_names if "Code_History_Database" in f), None)
    if db_file_name:
        # Try to extract the month and year from the bracketed part
        match = re.search(r"\(([^)]+)\)", db_file_name)
        if match:
            raw_date_str = match.group(1).strip().replace("_", " ")  # Fix underscores like "December_2023" -> "December 2023"
            try:
                parsed_date = datetime.strptime(raw_date_str, "%B %Y")
                database_date = parsed_date.strftime("%Y-%m")
                print(f"📅 Extracted database date: {database_date}")
            except ValueError:
                print("⚠️ Could not parse date string:", raw_date_str)
                database_date = None
        else:
            print("⚠️ No date in brackets found in filename.")
            database_date = None
    else:
        print("⚠️ Could not find a 'Code_History_Database' file in ZIP.")
        database_date = None

    # Extract Changes.csv and ChangeHistory.csv
    extracted_files = [f for f in zip_file_names if f.endswith("Changes.csv") or f.endswith("ChangeHistory.csv")]
    if not extracted_files:
        raise Exception("Neither Changes.csv nor ChangeHistory.csv was found in the ZIP file.")

    for file_name in extracted_files:
        print(f"Extracting {file_name} to {output_dir}/")
        with z.open(file_name) as source_file:
            destination_path = os.path.join(output_dir, os.path.basename(file_name))
            with open(destination_path, 'wb') as target_file:
                target_file.write(source_file.read())

print("✅ Both files downloaded and extracted to gsscoder_python/lookups")


📅 Extracted database date: 2023-12
Extracting ChangeHistory.csv to gsscoder_python/lookups/
Extracting Changes.csv to gsscoder_python/lookups/
✅ Both files downloaded and extracted to gsscoder_python/lookups


In [ ]:
database_date

'2023-12'

In [ ]:
# === Step 2: Read input files ===

changes_path = "gsscoder_python/lookups/Changes.csv"
history_path = "gsscoder_python/lookups/ChangeHistory.csv"

code_changes = pd.read_csv(changes_path, dtype=str)
change_history = pd.read_csv(history_path, dtype=str)

In [ ]:
# === Step 2: Prompt for and validate Code History Database Date ===

database_date_input = database_date

# Ensure date input is in correct format
while not re.match(r"^\d{4}-\d{2}$", database_date_input):
    database_date_input = input("Unexpected format for Code History Database Date entered. Please try again with format yyyy-mm: ")

# Assume end of the month for date
database_date = pd.to_datetime(f"{database_date_input}-31")

# Check for implausible dates
if database_date > pd.Timestamp.today():
    warnings.warn("The Code History Database Date given is in the future. This is unlikely to be correct, please double check.")

if database_date < pd.Timestamp("2009-01-01"):
    sys.exit("The Code History Database Date cannot be any earlier than 2009-01-01")

In [ ]:
# === Step 3: LAD filter ===

geogs_of_interest = ["E06", "E07", "E08", "E09", "W06"]

# Keep rows where ENTITYCD and GEOGCD_P are both LAD-level
lad_code_changes = code_changes[code_changes["ENTITYCD"].isin(geogs_of_interest)].copy()
lad_code_changes["old_entity"] = lad_code_changes["GEOGCD_P"].str[:3]
lad_code_changes = lad_code_changes[lad_code_changes["old_entity"].isin(geogs_of_interest)]

# Drop Welsh name columns
lad_code_changes.drop(columns=["GEOGNMW", "GEOGNMW_P"], inplace=True)

In [ ]:
# === Step 4: Detect Merges and Splits ===

# Merges: Multiple rows with same GEOGCD (new code)
merges = lad_code_changes["GEOGCD"].value_counts()
merges = merges[merges > 1].index.tolist()

# Splits: Multiple rows with same GEOGCD_P (old code)
splits = lad_code_changes["GEOGCD_P"].value_counts()
splits = splits[splits > 1].index.tolist()

lad_code_changes["split"] = lad_code_changes["GEOGCD_P"].isin(splits)
lad_code_changes["merge"] = lad_code_changes["GEOGCD"].isin(merges)

# Parse operational date
lad_code_changes["OPER_DATE"] = pd.to_datetime(lad_code_changes["OPER_DATE"], format="%d/%m/%Y")

# Rename and select relevant columns
lad_code_changes = lad_code_changes.rename(columns={
    "GEOGCD": "changed_to_code",
    "GEOGNM": "changed_to_name",
    "GEOGCD_P": "changed_from_code",
    "GEOGNM_P": "changed_from_name",
    "SI_TITLE": "desc",
    "ENTITYCD": "entity_type",
    "OPER_DATE": "date",
    "YEAR": "year"
})[[
    "changed_to_code", "changed_to_name", "changed_from_code", "changed_from_name",
    "desc", "entity_type", "date", "year", "split", "merge"
]]

In [ ]:
# === Step 5: Handle Minor Boundary Changes ===

minor_changes = {
    ("E08000020", "E06000057"),  # Gateshead → Northumberland
    ("E07000097", "E07000243"),  # East Hertfordshire → Stevenage
    ("W06000007", "W06000024"),  # Powys → Merthyr Tydfil
}
minor_descs = [
    "The Gateshead and Northumberland (Boundary Change) Order 2013",
    "The East Hertfordshire and Stevenage (Boundary Change) Order 2013",
    "The Merthyr Tydfil and Powys (Areas) Order 2009"
]

# Drop specific boundary transfer rows
lad_code_changes = lad_code_changes[
    ~lad_code_changes[["changed_from_code", "changed_to_code"]].apply(tuple, axis=1).isin(minor_changes)
]

# Set split and merge = False for rows with matching descriptions
lad_code_changes.loc[lad_code_changes["desc"].isin(minor_descs), ["split", "merge"]] = False

# Warn if new splits exist that aren't in minor list
new_splits = lad_code_changes[lad_code_changes["split"] == True]
if not new_splits.empty:
    print(new_splits)
    print("\nWARNING: New splits found that may require manual review.")

In [ ]:
# === Step 6: Prepare all LAD code history table ===

change_history = change_history[change_history["ENTITYCD"].isin(geogs_of_interest)].copy()

change_history["start_date"] = pd.to_datetime(change_history["OPER_DATE"], format="%d/%m/%Y", errors="coerce")
change_history["end_date"] = pd.to_datetime(change_history["TERM_DATE"], format="%d/%m/%Y", errors="coerce")

all_lad_codes_dates = change_history.rename(columns={
    "GEOGCD": "gss_code",
    "GEOGNM": "gss_name",
    "SI_TITLE": "desc",
    "ENTITYCD": "entity_type",
    "STATUS": "status"
})[[
    "gss_code", "gss_name", "desc", "entity_type", "start_date", "end_date", "status"
]]


In [ ]:
# === Step 7: Save Output Files ===

os.makedirs("gsscoder_python/lookups", exist_ok=True)
lad_code_changes.to_pickle("gsscoder_python/lookups/lad_code_changes.pickle")  # Py equivalent of RDS
all_lad_codes_dates.to_pickle("gsscoder_python/lookups/all_lad_codes_dates.pickle")
with open("gsscoder_python/lookups/database_date.txt", "w") as f:
    f.write(str(database_date))

print("✅ Lookup tables saved.")

PermissionError: [Errno 13] Permission denied: 'gsscoder_python'